In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
resnet = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/Jingyu/.torch/models/resnet18-5c106cde.pth
100.0%


In [6]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [ ]:
transform_test = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_train = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.RandomHorizontalFlip(p=0.2),
     transforms.RandomRotation(degrees=8),
     transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_batch_num = 100
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batch_num, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=train_batch_num, shuffle=True, num_workers=2)

resnet = torchvision.models.resnet18(pretrained=True)
resnet.fc = nn.Linear(1000, 100)
resnet.cuda()

# net = torch.load('net_L.pkl')

criterion = nn.CrossEntropyLoss()
LR = 0.001
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)


#####TRAINGING
for epoch in range(20):
    scheduler.step()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        if (epoch > 6):
            for group in optimizer.param_groups:
                for p in group['params']:
                    state = optimizer.state[p]
                    if (state['step'] >= 1024):
                        state['step'] = 1000
        optimizer.step()
    #     # print statistics
    #     # running_loss += loss.item()
    #     _, predicted = torch.max(outputs.data, 1)
    #     predicted_np = predicted.cpu().numpy()
    #     labels_np = labels.numpy()
    #     accuracy = (predicted_np == labels_np).sum() / float(train_batch_num)
    #     train_accu.append(accuracy)
    # accuracy_epoch = np.mean(train_accu)
    # print(epoch, accuracy_epoch)

    print('Epoch ', epoch, ' Finished Training')

    if epoch % 4 == 0:
        # print test accuracy after 3 epoch
        correct = 0
        total = 0
        # with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels_cuda = Variable(images.cuda()), Variable(labels.cuda())
            outputs = resnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels_cuda.size(0)
            predicted_np = predicted.cpu().numpy()
            labels_np = labels.numpy()

            correct += (predicted_np == labels_np).sum()

        print('Test Accuracy: %d %%' % (
                100 * correct / total))

#####TESTING
correct = 0
total = 0
# with torch.no_grad():
for data in testloader:
    images, labels = data
    images, labels_cuda = Variable(images.cuda()), Variable(labels.cuda())
    outputs = resnet(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels_cuda.size(0)
    predicted_np = predicted.cpu().numpy()
    labels_np = labels.numpy()

    correct += (predicted_np == labels_np).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

torch.save(resnet,'net_ljy.pkl')